In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

In [2]:
# Read csv into DataFrame
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop ID columns
application_df.drop(columns=["EIN","NAME"],inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Count the unique values in each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Value counts for APPLICATION_TYPE column
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# List of application types to replace with "Other"
application_types_to_replace = app_type_counts.loc[lambda x: x < 500].index
application_types_to_replace

Index(['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object', name='APPLICATION_TYPE')

In [7]:
# Replace
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app,"Other")

# Check if replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Value counts for CLASSIFICATION column
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# List of classifications to replace with "Other"
classifications_to_replace = classification_counts.loc[lambda x: x < 1000].index
classifications_to_replace

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='object', name='CLASSIFICATION')

In [10]:
# Replace
for classif in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(classif,"Other")

# Check if replacement was successful
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric
numeric_df = pd.get_dummies(application_df)
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  bool 
 4   APPLICATION_TYPE_T10          34299 non-null  bool 
 5   APPLICATION_TYPE_T19          34299 non-null  bool 
 6   APPLICATION_TYPE_T3           34299 non-null  bool 
 7   APPLICATION_TYPE_T4           34299 non-null  bool 
 8   APPLICATION_TYPE_T5           34299 non-null  bool 
 9   APPLICATION_TYPE_T6           34299 non-null  bool 
 10  APPLICATION_TYPE_T7           34299 non-null  bool 
 11  APPLICATION_TYPE_T8           34299 non-null  bool 
 12  AFFILIATION_CompanySponsored  34299 non-null  bool 
 13  AFFILIATION_Family/Parent     3

In [12]:
# Split dataframe into features and target arrays
y = numeric_df["IS_SUCCESSFUL"]
X = numeric_df.drop(columns="IS_SUCCESSFUL")

# Split features and target into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [13]:
# Instantiate StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training features
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=43,activation="relu",input_dim=43))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=20,activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1,activation="tanh"))

# Check model structure
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 43)                1892      
                                                                 
 dense_1 (Dense)             (None, 20)                880       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2793 (10.91 KB)
Trainable params: 2793 (10.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-07-30 14:48:58.371109: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-07-30 14:48:58.371130: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-07-30 14:48:58.371135: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-07-30 14:48:58.371166: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-30 14:48:58.371180: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# Compile the model
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [16]:
# Train the model
model.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
  1/804 [..............................] - ETA: 3:57 - loss: 5.8199 - accuracy: 0.4062

2023-07-30 14:48:58.635166: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 4s 5ms/step - loss: 2.0648 - accuracy: 0.6559
Epoch 2/25
804/804 [==============================] - 4s 5ms/step - loss: 1.7440 - accuracy: 0.6716
Epoch 3/25
804/804 [==============================] - 4s 5ms/step - loss: 1.7217 - accuracy: 0.6760
Epoch 4/25
804/804 [==============================] - 4s 5ms/step - loss: 1.7163 - accuracy: 0.6660
Epoch 5/25
804/804 [==============================] - 4s 5ms/step - loss: 1.7439 - accuracy: 0.6569
Epoch 6/25
804/804 [==============================] - 4s 5ms/step - loss: 1.5762 - accuracy: 0.6447
Epoch 7/25
804/804 [==============================] - 4s 5ms/step - loss: 2.9361 - accuracy: 0.5703
Epoch 8/25
804/804 [==============================] - 4s 5ms/step - loss: 0.8157 - accuracy: 0.6865
Epoch 9/25
804/804 [==============================] - 4s 5ms/step - loss: 0.7581 - accuracy: 0.6999
Epoch 10/25
804/804 [==============================] - 4s 5ms/step - loss: 2.7461 - accuracy: 0.5851
Epoch 11/2

In [17]:
# Evaluate the model using test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2023-07-30 14:50:33.903985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 1.3082 - accuracy: 0.6377 - 742ms/epoch - 3ms/step
Loss: 1.3081653118133545, Accuracy: 0.6376676559448242


In [18]:
# Export the model
model.save("model1.h5")

/Users/vinnyshankar/anaconda3/envs/p4/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Define the model
model2 = tf.keras.models.Sequential()

# First hidden layer
model2.add(tf.keras.layers.Dense(units=100,activation="relu",input_dim=43))

# Third hidden layer
model2.add(tf.keras.layers.Dense(units=50,activation="tanh"))

# Fourth hidden layer
model2.add(tf.keras.layers.Dense(units=25,activation="tanh"))

# Output layer
model2.add(tf.keras.layers.Dense(units=1,activation="tanh"))

# Check model structure
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               4400      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 25)                1275      
                                                                 
 dense_6 (Dense)             (None, 1)                 26        
                                                                 
Total params: 10751 (42.00 KB)
Trainable params: 10751 (42.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
model2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [21]:
# Train the model
model2.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
  1/804 [..............................] - ETA: 3:44 - loss: 2.2300 - accuracy: 0.5000

2023-07-30 14:50:34.862292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 4s 5ms/step - loss: 1.3359 - accuracy: 0.6811
Epoch 2/25
804/804 [==============================] - 4s 5ms/step - loss: 0.9306 - accuracy: 0.6861
Epoch 3/25
804/804 [==============================] - 4s 5ms/step - loss: 0.6439 - accuracy: 0.7091
Epoch 4/25
804/804 [==============================] - 4s 5ms/step - loss: 0.6013 - accuracy: 0.7194
Epoch 5/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5846 - accuracy: 0.7257
Epoch 6/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5891 - accuracy: 0.7256
Epoch 7/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5968 - accuracy: 0.7208
Epoch 8/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5841 - accuracy: 0.7225
Epoch 9/25
804/804 [==============================] - 4s 5ms/step - loss: 0.6472 - accuracy: 0.7148
Epoch 10/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5858 - accuracy: 0.7175
Epoch 11/2

In [22]:
# Evaluate the model using test data
model_loss, model_accuracy = model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2023-07-30 14:52:17.000295: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5662 - accuracy: 0.7269 - 761ms/epoch - 3ms/step
Loss: 0.5662127137184143, Accuracy: 0.7268804907798767


In [23]:
# Export the model
model2.save("model2.h5")

In [24]:
# Auto Optimizer Model Function
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    activation = hp.Choice("activation",["relu","tanh","sigmoid"])

    nn_model.add(tf.keras.layers.Dense(units=hp.Int("first_units",
                                                    min_value=1,
                                                    max_value=100,
                                                    step=2),
                                                    activation=activation,
                                                    input_dim=43))
    
    for i in range(hp.Int("num_layers",1,6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i),
                                                        min_value=1,
                                                        max_value=50,
                                                        step=2),
                                                        activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy",
                     optimizer="adam",
                     metrics=["accuracy"])
    
    return nn_model

In [25]:
# Tuner
tuner = kt.Hyperband(create_model,
                     objective="val_accuracy",
                     max_epochs=5,
                     hyperband_iterations=2,
                     overwrite=True)

In [26]:
# Run the Auto Optimizer
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test))

Trial 20 Complete [00h 00m 24s]
val_accuracy: 0.7162682414054871

Best val_accuracy So Far: 0.726064145565033
Total elapsed time: 00h 05m 43s
INFO:tensorflow:Oracle triggered exit


In [27]:
# Hyperparameters of best model found by auto optimizer
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 41,
 'num_layers': 5,
 'units_0': 23,
 'units_1': 31,
 'units_2': 5,
 'units_3': 37,
 'units_4': 19,
 'units_5': 33,
 'tuner/epochs': 5,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0014'}

In [28]:
# Results from best model found by auto optimizer
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2023-07-30 14:58:01.079311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5537 - accuracy: 0.7261 - 925ms/epoch - 3ms/step
Loss: 0.553662896156311, Accuracy: 0.726064145565033


In [29]:
best_model.save("model3.h5")

In [30]:
loaded_model = tf.keras.models.load_model("model3.h5")
loaded_model.get_config()

{'name': 'sequential',
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 43),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 43),
    'units': 41,
    'activation': 'tanh',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'registered_name': None,
   'build_config': {'inpu